### Import des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings


from arch import arch_model
    
from datetime import datetime, timedelta
#import datetime

# import tensorflow as tf

from numpy import array

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import arma_order_select_ic

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

from statsmodels.tsa.holtwinters import SimpleExpSmoothing 
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from statsmodels.graphics.tsaplots import plot_pacf

import os
import csv

import math

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

### Classe Loader

Classe pour charger les valeurs mobilières.



In [2]:
class Loader :

    # 1.ajouter vos valeurs mobilières ici
    ADA = "Cardano"
    AAVE = "Aave"
    BTC = "Bitcoin"
    COSMOS = "Cosmos"
    DOGECOIN = "Dogecoin"
    ETH = "Ethereum"
    LITECOIN = "Litecoin"
    MONERO = "Monero"
    POLKADOT = "Polkadot"
    USDCOIN = "USDCoin"

    MICROSOFT = "Microsoft"
    ATT = "AT&T"
    CAC40 = "cac40"

    # 2. ajouter la variable dans la liste
    SECURITY = [ADA, AAVE, BTC, COSMOS, DOGECOIN, ETH, LITECOIN, MONERO, POLKADOT, USDCOIN,
                MICROSOFT, ATT, CAC40]

    # 3. ajouter votre source ici (dossier `nouvelleSource` dans `data`)
    COINCODEX = "Coincodex"
    YAHOO = "finance.yahoo.com"
    KAGGLE = "Kaggle"
    NASDAQ = "Nasdaq"
    BINANCE = "Binance"

    # 4. ajouter votre source dans la liste
    SOURCES = [BINANCE, COINCODEX, YAHOO, KAGGLE, NASDAQ]

    # 5. ajouter votre nom de base de fichier pour la nouvelle source (la clé est la source)
    BASE_NAME = {
        BINANCE : "",
        COINCODEX : "coin_",
        YAHOO : "",
        KAGGLE : "coin_",
        NASDAQ : ""
    }

    # Granularity Day, Hour
    GRANULARIY = ['', 'h']

    def __init__(self):
        self.data = {}

        for source in Loader.SOURCES :
            self.data[source] = {}

    def loadFile(self, source, security, granularity='', dateStart=None, dateEnd=None):
        '''
        @param granulariy: default is day. Possible value is 'h'
        
        '''
        fileName = Loader.BASE_NAME[source]+security
        
        if(granularity != ''):
            fileName = fileName+'-'+granularity
        
        fileName = fileName+".csv"

        path = os.path.join("data", source, fileName)

#         print(path)
        
        if(os.path.isfile(path)):
            df = pd.read_csv(path,
                                header=0)
            
            df["Date"] = pd.to_datetime(df["Date"])
            df["Date"] = df["Date"].dt.date
            df= df.set_index("Date")

            df.sort_index(axis=0, inplace=True)
            
#             print(source, " - ", df.columns)
            
            if source == Loader.BINANCE:
#                 print("---------------", df.columns)
                df = df[['high', 'low', 'open', 'Close', 'Volume']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume']
                df['Market Cap'] = None
            elif source == Loader.COINCODEX:
                df = df[['High', 'Low', 'Open', 'Close', 'Volume', 'Market Cap']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap']
            elif source == Loader.KAGGLE:
                df = df[['High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap']
            elif source == Loader.NASDAQ:
                # ['Close/Last', 'Volume', 'Open', 'High', 'Low']
                df = df[['High', 'Low', 'Open', 'Close/Last', 'Volume']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume']
            elif source == Loader.YAHOO:
                # ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

                df = df[['High', 'Low', 'Open', 'Close', 'Volume']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume']

            if(dateStart and dateEnd):
                dateStart = datetime.strptime(dateStart, '%Y-%m-%d').date()
                dateEnd = datetime.strptime(dateEnd, '%Y-%m-%d').date()
                df = df.loc[dateStart:dateEnd]

            self.data[source][security] = df
#             print(source, "[", security, "]"," charged")

#         else:
#              print(source, "[", security, "]"," not charged")


    def loadSource(self, source, dateStart=None, dateEnd=None, forceEqualDate=False):

#         print("-----------------")

        for security in Loader.SECURITY:
                self.loadFile(source, security, dateStart=dateStart, dateEnd=dateEnd)

#         print("-----------------", ((dateStart and dateEnd) or forceEqualDate))

        if( (dateStart and dateEnd) or forceEqualDate):
            startDates = []        
            endDates =  []

            for security, df in self.data[source].items():
                startDates.append(df.index[0])
                endDates.append(df.index[-1])

            s = max(startDates)
            e = min(endDates)

            for security, df in self.data[source].items():
                self.data[source][security] = df.loc[s:e]
        
    def loadSources(self, dateStart=None, dateMax=None):
        for source in Loader.SOURCES:
            self.loadSource(source)

    def get(self, source, nameSecurity):
        return self.data[source][nameSecurity]



### Exemple d'utilisation

Pour charger  
 une données précises, il faut indiquer :

- la source des données (un dossier par source dans le dossier `data`)
- le nom de la valeur mobilière
- en option, les dates sur lesquels charger la donnée



In [3]:
loader = Loader()

loader.loadFile(Loader.COINCODEX, Loader.ADA, dateStart="2000-01-01", dateEnd="2024-01-01")

df = loader.get(Loader.COINCODEX, Loader.ADA)


NameError: name 'Loader' is not defined

Pour charger une source, il faut indiquer :
- la source des données (un dossier par source dans le dossier `data`)
- en option, les dates sur lesquels charger la donnée

In [4]:

loader.loadSource(Loader.COINCODEX, dateStart="2000-01-01", dateEnd="2024-01-01")

# loader.data

NameError: name 'loader' is not defined

In [11]:
loader.loadSource(Loader.COINCODEX, forceEqualDate=True)


-----------------
data/Coincodex/coin_Cardano.csv
Coincodex  -  Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'], dtype='object')
Coincodex [ Cardano ]  charged
data/Coincodex/coin_Aave.csv
Coincodex [ Aave ]  not charged
data/Coincodex/coin_Bitcoin.csv


Coincodex  -  Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'], dtype='object')
Coincodex [ Bitcoin ]  charged
data/Coincodex/coin_Cosmos.csv
Coincodex [ Cosmos ]  not charged
data/Coincodex/coin_Dogecoin.csv
Coincodex [ Dogecoin ]  not charged
data/Coincodex/coin_Ethereum.csv


Coincodex  -  Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'], dtype='object')
Coincodex [ Ethereum ]  charged
data/Coincodex/coin_Litecoin.csv
Coincodex [ Litecoin ]  not charged
data/Coincodex/coin_Monero.csv
Coincodex [ Monero ]  not charged
data/Coincodex/coin_Polkadot.csv
Coincodex [ Polkadot ]  not charged
data/Coincodex/coin_USDCoin.csv
Coincodex [ USDCoin ]  not charged
data/Coincodex/coin_Microsoft.csv
Coincodex [ Microsoft ]  not charged
data/Coincodex/coin_AT&T.csv
Coincodex [ AT&T ]  not charged
data/Coincodex/coin_cac40.csv
Coincodex [ cac40 ]  not charged
----------------- True


In [12]:

dfAda = loader.data[Loader.COINCODEX][Loader.ADA]
dfBtc = loader.data[Loader.COINCODEX][Loader.BTC]

# print("Ada from ", dfAda.index[0], " to ", dfAda.index[-1])
# print("Btc from ", dfBtc.index[0], " to ", dfBtc.index[-1])

In [7]:
# loader.loadSource(Loader.BINANCE, forceEqualDate=True)


In [8]:

# dfAda = loader.data[Loader.BINANCE][Loader.ADA]

# print("Ada from ", dfAda.index[0], " to ", dfAda.index[-1])


Pour charger toutes les données :

In [9]:
# loader.loadSources()

### Binance API - non fonctionnel

binanceAPI = BinanceAPI()

binanceAPI.saveCoin("BTC")

client = Client()
klines = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jan, 2000")


header = ['Unix','Date','Symbol','Open','High','Low','Close','Volume','Close time','Quote asset volume','Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'can be ignored']
with open( CoinSymbol+'.csv', 'w' ,newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for ligne in COIN:
        ligne.insert(1,datetime.fromtimestamp(ligne[0]/1000))
        ligne.insert(2,CoinSymbol)
        writer.writerow(ligne)


print(len(klines))
klines

In [10]:
# class BinanceAPI:

#     def __init__(self, granularity=Client.KLINE_INTERVAL_1DAY):
#         self.client = Client()
#         self.path = os.path.join('data', 'Binance')
#         self.granularity = granularity


#     def saveCoin(self, key):
#         data = self.client.get_historical_klines(key+"USDT", self.granularity, "1 Jan, 2000")
#         fileName = os.path.join(self.path, key+".csv")

#         header = ['Unix','Date','Symbol','Open','High','Low','Close','Volume','Close time','Quote asset volume','Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'can be ignored']

#         with open( fileName, 'w' ,newline='') as f:
#             writer = csv.writer(f)
#             writer.writerow(header)
#             for ligne in data:
#                 ligne.insert(1, datetime.fromtimestamp(ligne[0]/1000))
#                 ligne.insert(2, key)
#                 writer.writerow(ligne)